1. Создаем ретривер на chroma и  bge-small-en-v1.5
2. Делаем RAG с моделью LLM через библиотку Hugging Face. Возможность получить от модели вариативный вывод есть только transformers, поэтому мы сильно ограничены. В этой тетрадке я использую модель gtp2. У меня почему-то локально не хочет скачивать кватизированные модели. 
3. Создает пустой графовый ретривер, добавляя в него заранее сгенерированные триплеты. 
4. Создаем KG-ретривер, котрорый будет возвращать релевантные к запросу триплеты.

Какая логика:
1. Мы отправляем запрос в RAG-сеттинг и получаем от модели несколько ответов. 
2. Отправляем этот же запрос в KG-ретривер, чтобы получить релеватные триплеты.
3. Реранк Ответов модели:
 * Первый вариант: Получаем векторное представление найденных триплетов и сравниваем его с эмббедингами ответов. Предполагаем, что эмббединг наилучшего ответа модели будет наиболее близок к эмббедингу триплетов. 
 * Второй вариант: Просим большую модель с учетом триплетов реранжировать ответы изначальной модели. 

In [1]:
import json
import chromadb
from llama_index.core import ServiceContext, StorageContext
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from llama_index.llms.openai import OpenAI as OpenAIllm
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import os
import ast
import torch
import yaml
import openai
from openai import OpenAI
from tqdm.auto import tqdm

set_seed(42)

/Users/dima/Repo/Alexandra/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Параметры скрипта

DOMAIN = 'movie'
# DOMAIN = 'computer'
# DOMAIN = 'nature'

In [3]:
with open('secrets.yaml', 'r') as f:
    secrets = yaml.safe_load(f)

openai_key = secrets['openai_key']

os.environ['OPEN_AI_KEY'] = openai_key
openai.api_key = openai_key

Загружаем данные

In [4]:
dataset_path = os.path.join('artifacts', DOMAIN, 'eval_dataset.csv')
df = pd.read_csv(dataset_path)
df.head()

,sent,question,rag_0,rag_1,rag_2,uniq_count
0,Chapleau River is in the James Bay drainage ba...,What is the main tributary of Kapuskasing Lake?,Chapleau River,Chapleau River is the main tributary of Kapusk...,Chapleau River,2
1,The mountains classification was won by Nicola...,Who won the mountains classification in the To...,Juan Felipe Osorio won the mountains classific...,Juan Felipe Osorio won the mountains classific...,Juan Felipe Osorio won the mountains classific...,2
2,"The wreathed hornbill (Rhyticeros undulatus), ...",What is the distribution of the wreathed hornb...,The wreathed hornbill is found in forests from...,The wreathed hornbill species is found in fore...,The distribution of the wreathed hornbill spec...,3
3,The podium placings were completed by another ...,Who finished third on Willunga Hill and took f...,Tom-Jelte Slagter of Team Dimension Data.,Tom-Jelte Slagter of Team Dimension Data finis...,Tom-Jelte Slagter of Team Dimension Data.,2
4,The Australian swiftlet (Aerodramus terraeregi...,What is the scientific name of the Australian ...,Aerodramus terraereginae,Aerodramus terraereginae.,The scientific name of the Australian swiftlet...,3


Иницилизируем модель для получения эмббедингов

In [7]:
embed_model = SentenceTransformerEmbeddingFunction(model_name="BAAI/bge-small-en-v1.5", 
                                                device='cpu'
                                                )

## RAG QA

В отдельном файле rag_cloud.ipynb

## KG ретривер

Делаем три ретривера для трех графов: fine tuned, fine tuned postprocessed, ground truth.<br>
Cохраняем в query_engines

Если требуется, то комментарии ретривера есть в сходном файле rag_rerank.ipynb

In [8]:
kg_datases = {
    'ft': os.path.join('artifacts', DOMAIN, 'triples_ft.jsonl'),
    'ftpp': os.path.join('artifacts', DOMAIN, 'triples_ft_pp.jsonl'),
    'gt': os.path.join('artifacts', DOMAIN, 'triples_gt.jsonl'),
}

query_engines = {}

for k, file_path in kg_datases.items():

    with open(file_path, "r") as f:
        data = f.readlines()
        
    triplets = []
    for i_line in data:
        i_line = json.loads(i_line)
        for i_triplet in i_line['triples']:
            if 'sub' in i_triplet and 'rel' in i_triplet and 'obj' in i_triplet:
                prep_triplet = [i_triplet['sub'], i_triplet['rel'], i_triplet['obj']]
                triplets.append(prep_triplet)

    llm = OpenAIllm(temperature=0, 
                #model="gpt-4-1106-preview", 
                openai_api_key = openai_key)

    graph_store = SimpleGraphStore()

    for i_triplet in triplets:
        graph_store.upsert_triplet(i_triplet[0], i_triplet[1], i_triplet[2])

    storage_context = StorageContext.from_defaults(graph_store=graph_store)

    graph_rag_retriever = KnowledgeGraphRAGRetriever(
        storage_context=storage_context,
        verbose=True
    )

    query_engine = RetrieverQueryEngine.from_args(
        graph_rag_retriever,
    )

    query_engines[k] = query_engine


## Реранжирование ответов модели 

### Первый вариант

In [9]:
def rerank_outputs_cosine(model_outputs, kg_response):
    """
    Функция для переранжировки ответов модели на основе косинусного сходства с триплетами из базы знаний.

    Аргументы:
    - model_outputs: список ответов модели
    - kg_response: ответ из базы знаний

    Возвращает:
    - Отсортированный список текстовых выводов на основе косинусного сходства с триплетами из базы знаний.
    """
    if kg_response.source_nodes:
        triplets_set = set()
        for i_triplet in kg_response.source_nodes[0].metadata['kg_rel_text']:
            triplets_set.update(set(ast.literal_eval(i_triplet)))
        triplets_string = ' '.join(triplets_set)
        # triplets_string

        triplets_emb = embed_model(triplets_string)[0]
        outputs_emb = embed_model(model_outputs)

        similarities = cosine_similarity(outputs_emb, [triplets_emb])
        # print(similarities)

        sorted_indices = np.argsort(-similarities[:, 0]) 

        sorted_texts = [model_outputs[index] for index in sorted_indices]
        return sorted_texts
    
    # return "Не удалось найти триплеты."
    # если нет триплетов, то означает, что улучшить не получилось, ответ не меняется
    return model_outputs

Проходим по датасету. Реранжируем только различающиеся ответы

In [10]:
success_cnt = {'ft': 0, 'ftpp': 0, 'gt': 0}

df['cosine_reranked_ft'] = None
df['cosine_reranked_ftpp'] = None
df['cosine_reranked_gt'] = None

for index in tqdm(df[df.uniq_count > 1].index):
    query = df.loc[index, 'question']
    output_texts = [df.loc[index, 'rag_0'], df.loc[index, 'rag_1'], df.loc[index, 'rag_2']]

    for k, query_engine in query_engines.items():
        response = query_engine.query(query)
        if response.source_nodes:
            reranked_outputs = rerank_outputs_cosine(output_texts, response)
            success_cnt[k] += 1
        else:
            reranked_outputs = output_texts
        df.loc[index, f'cosine_reranked_{k}'] = reranked_outputs[0]

df.to_csv(dataset_path, index=False)

print(f'Найдены знания в {success_cnt}')

100%|██████████| 114/114 [09:04<00:00,  4.77s/it]

Найдены знания в {'ft': 40, 'ftpp': 33, 'gt': 63}


In [11]:
print('Количесво изменений ответов:')
for k in query_engines.keys():
    x = ((df.uniq_count > 1) & (df.rag_0 != df[f'cosine_reranked_{k}'])).sum()
    print(f'{k}: {x}')



Количесво изменений ответов:
k: 20
k: 16
k: 34


### Второй вариант

In [12]:
client = OpenAI(api_key=openai_key)

In [13]:
rerank_template = """You are provided a question and list of answers. You are to rank the answers in order of relevance to the facts in provided triplets. 
If an answer is not relevant to the triplets and question, rank it last. If an answer is relevant to the question and triplets, rank it based on how relevant it is.

Your output must be a list of the answers in order of relevance to the question and triplets.
####################
Question: {question}
Triplets: {triplets}
Answers: {answers}
####################"""

In [14]:
def rerank_outputs_llm(query, model_outputs, kg_response, model="gpt-3.5-turbo"):
    """
    Функция для переранжировки ответов модели на основе llm сходства с триплетами из базы знаний.

    Аргументы:
    - query: запрос
    - model_outputs: список ответов модели
    - kg_response: ответ из базы знаний

    Возвращает:
    - Отсортированный список текстовых выводов 
    """

    if kg_response.source_nodes:
        prompt = rerank_template.format(
            question=query, 
            triplets=kg_response.source_nodes[0].metadata['kg_rel_text'], 
            answers=model_outputs)

        gpt_response = client.chat.completions.create(
            model=model,
            messages=[
                {
                "role": "user",
                "content": prompt
                }
            ],
            temperature=0,
        )

        gpt_response_text = gpt_response.choices[0].message.content
        try:
            sorted_indices = [int(x[0]) for x in gpt_response_text.split('\n')[:3]]
            sorted_texts = [model_outputs[index-1] for index in sorted_indices]
            return sorted_texts            
        except:
            try:
                x = json.loads('{ "x":'+gpt_response_text.replace("'", '"')+'}')['x']
            except:
                x = None
            if x is not None and isinstance(x, list) and len(x) == 3:
                sorted_texts = x
            else:
                print('Error parsing GPT')
                print(gpt_response_text)
                print('')
                return model_outputs

    # return "Не удалось найти триплеты."
    # если нет триплетов, то означает, что улучшить не получилось, ответ не меняется
    return model_outputs

In [15]:
success_cnt = {'ft': 0, 'ftpp': 0, 'gt': 0}

df['llm_reranked_ft'] = None
df['llm_reranked_ftpp'] = None
df['llm_reranked_gt'] = None

for index in tqdm(df[df.uniq_count > 1].index):
    query = df.loc[index, 'question']
    output_texts = [df.loc[index, 'rag_0'], df.loc[index, 'rag_1'], df.loc[index, 'rag_2']]

    for k, query_engine in query_engines.items():
        response = query_engine.query(query)
        if response.source_nodes:
            reranked_outputs = rerank_outputs_llm(query, output_texts, response)
            success_cnt[k] += 1
        else:
            reranked_outputs = output_texts
        df.loc[index, f'llm_reranked_{k}'] = reranked_outputs[0]

df.to_csv(dataset_path, index=False)

print(f'Найдены знания в {success_cnt}')

100%|██████████| 114/114 [11:44<00:00,  6.18s/it]

Найдены знания в {'ft': 40, 'ftpp': 33, 'gt': 63}


In [16]:
print('Количесво изменений ответов:')
for k in query_engines.keys():
    x = ((df.uniq_count > 1) & (df.rag_0 != df[f'llm_reranked_{k}'])).sum()
    print(f'k: {x}')

Количесво изменений ответов:
k: 0
k: 0
k: 0
